Step 0: Import all relevant libraries used in this project

In [8]:
from sodapy import Socrata # for API
import pyodbc # for SQL
import pandas as pd # for data cleaning
import dbfread # for dbf transformation
from simpledbf import Dbf5 # for dbf transformation

Step 1: Download crime data related to CTA from the City of Chicago data portal API until November 15, 2019

In [4]:
## WARNING: this query takes approx. 5 minutes to run; don't run it everytime you run this script

# API instructions https://dev.socrata.com/foundry/data.cityofchicago.org/ijzp-q8t2
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
# client = Socrata("data.cityofchicago.org", None)

# Example authenticated client (needed for non-public datasets):
client = Socrata("data.cityofchicago.org",
                  "QtMhXqaTTglPlVS3AC6PEQQxD", username = "juli.kleindiek@gmail.com", password = "DEPA_2019")

# Limit to 6000 rows to avoid time out errors, data has <6000 rows, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
# Pull all crime data before '2019-11-15T00:00:00.000'
results = client.get("5xiy-qnsz", where = "date <= '2019-11-15T00:00:00.000'", limit = 6000)

Step 2: Clean the crime data

In [5]:
# Look at the output of the dowload
results[1]

{'date': '2011-01-01T01:30:00.000',
 'location': {'latitude': '41.883376682',
  'human_address': '{"address": "", "city": "", "state": "", "zip": ""}',
  'needs_recoding': False,
  'longitude': '-87.627874384'},
 'district': '001',
 'block': '001XX N STATE ST',
 'y_coordinate': '1900882',
 'latitude': '41.883376682',
 'description': 'SIMPLE',
 'location_description': 'CTA PLATFORM',
 'updated_on': '2016-02-04T06:33:39.000',
 'community_area': '32',
 'iucr': '0560',
 'x_coordinate': '1176353',
 'ward': '42',
 'case_number': 'HT100175',
 'year': '2011',
 'domestic': False,
 'fbi_code': '08A',
 'longitude': '-87.627874384',
 'beat': '0122',
 'primary_type': 'ASSAULT',
 'arrest': False,
 'id': '7869621'}

In [9]:
# Convert results to pandas DataFrame
crime_dirty = pd.DataFrame.from_records(results)
crime_dirty.head()

,arrest,beat,block,case_number,community_area,date,description,district,domestic,fbi_code,...,latitude,location,location_description,longitude,primary_type,updated_on,ward,x_coordinate,y_coordinate,year
0,True,0624,008XX E 79TH ST,HT100317,69,2011-01-01T00:15:00.000,SIMPLE,006,False,08B,...,41.751291477,"{'latitude': '41.751291477', 'human_address': ...",CTA BUS,-87.604904392,BATTERY,2016-02-04T06:33:39.000,6,1183016,1852803,2011
1,False,0122,001XX N STATE ST,HT100175,32,2011-01-01T01:30:00.000,SIMPLE,001,False,08A,...,41.883376682,"{'latitude': '41.883376682', 'human_address': ...",CTA PLATFORM,-87.627874384,ASSAULT,2016-02-04T06:33:39.000,42,1176353,1900882,2011
2,False,0724,011XX W 69TH ST,HT100156,68,2011-01-01T01:45:00.000,SIMPLE,007,False,08B,...,41.768727336,"{'latitude': '41.768727336', 'human_address': ...",CTA BUS,-87.652090375,BATTERY,2016-02-04T06:33:39.000,17,1170090,1859051,2011
3,False,0123,0000X S STATE ST,HT100176,32,2011-01-01T02:00:00.000,OVER $500,001,False,06,...,41.881556538,"{'latitude': '41.881556538', 'human_address': ...",CTA TRAIN,-87.627758535,THEFT,2016-02-04T06:33:39.000,42,1176390,1900219,2011
4,False,1812,009XX W FULLERTON AVE,HT101146,7,2011-01-01T02:00:00.000,POCKET-PICKING,018,False,06,...,41.925398449,"{'latitude': '41.925398449', 'human_address': ...",CTA TRAIN,-87.652311296,THEFT,2016-02-04T06:33:39.000,43,1169577,1916141,2011


In [10]:
# investigate crime_dirty 
crime_dirty.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5808 entries, 0 to 5807
Data columns (total 22 columns):
arrest                  5808 non-null bool
beat                    5808 non-null object
block                   5808 non-null object
case_number             5808 non-null object
community_area          5808 non-null object
date                    5808 non-null object
description             5808 non-null object
district                5808 non-null object
domestic                5808 non-null bool
fbi_code                5808 non-null object
id                      5808 non-null object
iucr                    5808 non-null object
latitude                5806 non-null object
location                5806 non-null object
location_description    5808 non-null object
longitude               5806 non-null object
primary_type            5808 non-null object
updated_on              5808 non-null object
ward                    5808 non-null object
x_coordinate            5806 non-null obje

In [11]:
# Validate the values , i.e. 'id'
crime_dirty.loc[2:3]['id']

2    7869647
3    7869682
Name: id, dtype: object

In [12]:
# bring dataframe into proper format
crime = crime_dirty[['id', 
        'case_number', 
        'date', 
        'block', 
        'iucr', 
        'primary_type', 
        'description', 
        'location_description',
        'arrest',
        'domestic',
        'beat',
        'district',
        'ward',
        'community_area',
        'fbi_code',
        'x_coordinate',
        'y_coordinate',
        'year',
        'updated_on',
        'latitude',
        'longitude']]

In [13]:
# take a look at the proper dataframe
crime.head()

,id,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,district,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude
0,7870071,HT100317,2011-01-01T00:15:00.000,008XX E 79TH ST,0460,BATTERY,SIMPLE,CTA BUS,True,False,...,006,6,69,08B,1183016,1852803,2011,2016-02-04T06:33:39.000,41.751291477,-87.604904392
1,7869621,HT100175,2011-01-01T01:30:00.000,001XX N STATE ST,0560,ASSAULT,SIMPLE,CTA PLATFORM,False,False,...,001,42,32,08A,1176353,1900882,2011,2016-02-04T06:33:39.000,41.883376682,-87.627874384
2,7869647,HT100156,2011-01-01T01:45:00.000,011XX W 69TH ST,0460,BATTERY,SIMPLE,CTA BUS,False,False,...,007,17,68,08B,1170090,1859051,2011,2016-02-04T06:33:39.000,41.768727336,-87.652090375
3,7869682,HT100176,2011-01-01T02:00:00.000,0000X S STATE ST,0810,THEFT,OVER $500,CTA TRAIN,False,False,...,001,42,32,06,1176390,1900219,2011,2016-02-04T06:33:39.000,41.881556538,-87.627758535
4,7870254,HT101146,2011-01-01T02:00:00.000,009XX W FULLERTON AVE,0870,THEFT,POCKET-PICKING,CTA TRAIN,False,False,...,018,43,7,06,1169577,1916141,2011,2016-02-04T06:33:39.000,41.925398449,-87.652311296


In [14]:
# rename column 'id' into 'crimeID'
crime = crime.rename(columns={"id": "crimeID"})

In [15]:
# define proper data types for each column; WE NEED FURTHER CLEANING HERE
crime = crime.astype({"crimeID": int})

In [16]:
# set index of crime dataframe to 'id'
crime.set_index('crimeID')

,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,beat,district,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude
crimeID,,,,,,,,,,,,,,,,,,,,
7870071,HT100317,2011-01-01T00:15:00.000,008XX E 79TH ST,0460,BATTERY,SIMPLE,CTA BUS,True,False,0624,006,6,69,08B,1183016,1852803,2011,2016-02-04T06:33:39.000,41.751291477,-87.604904392
7869621,HT100175,2011-01-01T01:30:00.000,001XX N STATE ST,0560,ASSAULT,SIMPLE,CTA PLATFORM,False,False,0122,001,42,32,08A,1176353,1900882,2011,2016-02-04T06:33:39.000,41.883376682,-87.627874384
7869647,HT100156,2011-01-01T01:45:00.000,011XX W 69TH ST,0460,BATTERY,SIMPLE,CTA BUS,False,False,0724,007,17,68,08B,1170090,1859051,2011,2016-02-04T06:33:39.000,41.768727336,-87.652090375
7869682,HT100176,2011-01-01T02:00:00.000,0000X S STATE ST,0810,THEFT,OVER $500,CTA TRAIN,False,False,0123,001,42,32,06,1176390,1900219,2011,2016-02-04T06:33:39.000,41.881556538,-87.627758535
7870254,HT101146,2011-01-01T02:00:00.000,009XX W FULLERTON AVE,0870,THEFT,POCKET-PICKING,CTA TRAIN,False,False,1812,018,43,7,06,1169577,1916141,2011,2016-02-04T06:33:39.000,41.925398449,-87.652311296
7869713,HT100235,2011-01-01T02:30:00.000,009XX W FULLERTON AVE,1563,SEX OFFENSE,CRIMINAL SEXUAL ABUSE,CTA TRAIN,False,False,1812,018,43,7,17,1169577,1916141,2011,2016-02-04T06:33:39.000,41.925398449,-87.652311296
7870065,HT100800,2011-01-01T02:40:00.000,0000X W 79TH ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,CTA BUS STOP,False,True,0623,006,6,44,08B,1177483,1852629,2011,2016-02-04T06:33:39.000,41.750940757,-87.625185222
7869656,HT100245,2011-01-01T02:50:00.000,015XX W CHICAGO AVE,0460,BATTERY,SIMPLE,CTA BUS,False,False,1323,012,1,24,08B,1165806,1905429,2011,2016-02-04T06:33:39.000,41.896085375,-87.666473716
7869657,HT100301,2011-01-01T03:30:00.000,008XX W NORTH AVE,0870,THEFT,POCKET-PICKING,CTA TRAIN,False,False,1822,018,43,8,06,1170480,1910867,2011,2016-02-04T06:33:39.000,41.910906593,-87.649147871


In [ ]:
# Further cleaning?


Step 3: Create Grid Table based on crime data

Step 4: Assign GridID to crime data

Step 5: Load BusStop and TrainStop data

Step 6: Clean BusStop and TrainStop data

Step 7: Assign GridID to BusStop and TrainStop data

Step 8: Load holiday data

Step 9: Clean holiday data

Step 10: Load tables into CloudSQL

In [ ]:
# define connection to the server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=server_name;'
                      'Database=db_name;'
                      'Trusted_Connection=yes;')

In [ ]:
# define connection to the server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=server_name;'
                      'Database=db_name;'
                      'Trusted_Connection=yes;')

Step 11: Daily refresh of crime data

In [17]:
# pull most recent date from table

latest_date = crime.date.max()
latest_date

'2011-12-30T23:55:00.000'

In [ ]:
# prepare API statement: filter for dates that are more recent than the max date in the table

updated_statement = "date" + " " + ">" + " " + "'" + latest_date + "'"
updated_statement

In [ ]:
# Pull all crime data before '2019-11-15T00:00:00.000'
client = Socrata("data.cityofchicago.org",
                  "QtMhXqaTTglPlVS3AC6PEQQxD", username = "juli.kleindiek@gmail.com", password = "DEPA_2019")

# Limit to 5 rows for test purposes
results = client.get("5xiy-qnsz", where = updated_statement, limit = 5)

In [ ]:
# Prepare new data (only example)
crime_new_dirty = pd.DataFrame.from_records(results)
crime_new = crime_new_dirty[['id', 
        'case_number', 
        'date', 
        'block', 
        'iucr', 
        'primary_type', 
        'description', 
        'location_description',
        'arrest',
        'domestic',
        'beat',
        'district',
        'ward',
        'community_area',
        'fbi_code',
        'x_coordinate',
        'y_coordinate',
        'year',
        'updated_on',
        'latitude',
        'longitude']]
crime_new.tail()

In [ ]:
# Append new data to master crime data file
crime.append(crime_new)

Step 12: Clean daily updated crime data

Step 13: Assign GridID to daily updated crime data

Step 14: Append daily updated crime data to the crime database in CloudSQL

Import and clean CTA data (for reference of .dbf to .csv transformation, see https://pypi.org/project/simpledbf/; perform $ pip install simpledbf)

In [22]:
# import CTA_BusStops.dbf
# retrieved from GitHub
# originally downloaded from https://data.cityofchicago.org/Transportation/CTA-Bus-Stops-Shapefile/pxug-u72f
dbf1 = Dbf5('Datasets/CTA_BusStops.dbf', codec='utf-8')

In [23]:
# take a look at the file
dbf1.fields

[('DeletionFlag', 'C', 1),
 ('OBJECTID', 'N', 10),
 ('SYSTEMSTOP', 'N', 19),
 ('STREET', 'C', 75),
 ('CROSS_ST', 'C', 75),
 ('DIR', 'C', 3),
 ('POS', 'C', 4),
 ('ROUTESSTPG', 'C', 75),
 ('OWLROUTES', 'C', 20),
 ('CITY', 'C', 20),
 ('STATUS', 'N', 10),
 ('PUBLIC_NAM', 'C', 75),
 ('POINT_X', 'N', 19),
 ('POINT_Y', 'N', 19)]

In [24]:
# export .dbf file to .csv (BusStops)
dbf1.to_csv('Datasets/CTA_BusStops.csv')

In [25]:
# read .csv for BusStops
BusStops = pd.read_csv('Datasets/CTA_BusStops.csv', index_col = 'OBJECTID')

In [26]:
# change column name POINT_X and POINT_Y to lat and long
BusStops = BusStops.rename(columns={"POINT_X": "longitude", "POINT_Y":"latitude"})

In [27]:
# look at clean data frame
BusStops.head()

,SYSTEMSTOP,STREET,CROSS_ST,DIR,POS,ROUTESSTPG,OWLROUTES,CITY,STATUS,PUBLIC_NAM,longitude,latitude
OBJECTID,,,,,,,,,,,,
193,6696.0,TAYLOR,THROOP,EB,NS,157,NaN,CHICAGO,1,Taylor & Throop,-87.659294,41.869314
194,22.0,JACKSON,KARLOV,EB,FS,126,NaN,CHICAGO,1,Jackson & Karlov,-87.727808,41.877007
195,4767.0,FOSTER,MONTICELLO,EB,NS,92,NaN,CHICAGO,1,Foster & Monticello,-87.719780,41.975526
196,6057.0,ASHLAND,CERMAK/BLUE ISLAND,SB,NS,"9,X9",N9,CHICAGO,1,Ashland & Cermak/Blue Island,-87.666173,41.852484
197,1790.0,CLARK,ALBION,SB,NS,22,N22,CHICAGO,1,Clark & Albion,-87.671981,42.001785


In [28]:
# import CTA_TrainStops.csv
# retrieved from GitHub
# originally downloaded from https://data.cityofchicago.org/Transportation/CTA-System-Information-List-of-L-Stops/8pix-ypme
TrainStops = pd.read_csv('Datasets/CTA_TrainStops.csv', index_col = 'STOP_ID')
TrainStops.head()

,DIRECTION_ID,STOP_NAME,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,ADA,RED,BLUE,G,BRN,P,Pexp,Y,Pnk,O,Location
STOP_ID,,,,,,,,,,,,,,,,
30162,W,18th (54th/Cermak-bound),18th,18th (Pink Line),40830,True,False,False,False,False,False,False,False,True,False,"(41.857908, -87.669147)"
30161,E,18th (Loop-bound),18th,18th (Pink Line),40830,True,False,False,False,False,False,False,False,True,False,"(41.857908, -87.669147)"
30022,N,35th/Archer (Loop-bound),35th/Archer,35th/Archer (Orange Line),40120,True,False,False,False,False,False,False,False,False,True,"(41.829353, -87.680622)"
30023,S,35th/Archer (Midway-bound),35th/Archer,35th/Archer (Orange Line),40120,True,False,False,False,False,False,False,False,False,True,"(41.829353, -87.680622)"
30214,S,35-Bronzeville-IIT (63rd-bound),35th-Bronzeville-IIT,35th-Bronzeville-IIT (Green Line),41120,True,False,False,True,False,False,False,False,False,False,"(41.831677, -87.625826)"


In [29]:
# clean up TrainStations's Location column into Point_X and Point_Y
# split location column
LocationNew = TrainStops["Location"].str.split(",", n = 1, expand = True)
LocationNew.head()

,0,1
STOP_ID,,
30162,(41.857908,-87.669147)
30161,(41.857908,-87.669147)
30022,(41.829353,-87.680622)
30023,(41.829353,-87.680622)
30214,(41.831677,-87.625826)


In [30]:
# remove parentheses
LocationNew[0].replace(regex=True,inplace=True,to_replace=r'\(',value=r'')
LocationNew[1].replace(regex=True,inplace=True,to_replace=r'\)',value=r'')
LocationNew.head()

,0,1
STOP_ID,,
30162,41.857908,-87.669147
30161,41.857908,-87.669147
30022,41.829353,-87.680622
30023,41.829353,-87.680622
30214,41.831677,-87.625826


In [31]:
# add Point_Y and Point_X to dataframe
TrainStops["latitude"] = LocationNew[0]
TrainStops["longitude"] = LocationNew[1]
TrainStops = TrainStops.drop("Location", 1)

In [32]:
# look at clean data frame
TrainStops.head()

,DIRECTION_ID,STOP_NAME,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,ADA,RED,BLUE,G,BRN,P,Pexp,Y,Pnk,O,latitude,longitude
STOP_ID,,,,,,,,,,,,,,,,,
30162,W,18th (54th/Cermak-bound),18th,18th (Pink Line),40830,True,False,False,False,False,False,False,False,True,False,41.857908,-87.669147
30161,E,18th (Loop-bound),18th,18th (Pink Line),40830,True,False,False,False,False,False,False,False,True,False,41.857908,-87.669147
30022,N,35th/Archer (Loop-bound),35th/Archer,35th/Archer (Orange Line),40120,True,False,False,False,False,False,False,False,False,True,41.829353,-87.680622
30023,S,35th/Archer (Midway-bound),35th/Archer,35th/Archer (Orange Line),40120,True,False,False,False,False,False,False,False,False,True,41.829353,-87.680622
30214,S,35-Bronzeville-IIT (63rd-bound),35th-Bronzeville-IIT,35th-Bronzeville-IIT (Green Line),41120,True,False,False,True,False,False,False,False,False,False,41.831677,-87.625826


Import and clead holiday data

In [ ]:
# @Lola: insert code for loading holiday data

Establie API to weather data

In [ ]:
# @Lola: insert code for for establishing API to weather data